# Content-based Model

## one-hot-vector

유사한 장르의 영화를 추천

input = 행 - 영화, 열 - 영화의 장르, 값 - 원핫벡터

output = 각 영화별 유사한 장르의 영화를 추천 (여러가지 유사도를 이용하여 구함)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
movie_paths = '/content/drive/MyDrive/추천/data/movies/'

meta = pd.read_csv(movie_paths + 'movies_metadata.csv', low_memory=False)
meta = meta[meta['overview'].notnull()].reset_index(drop=True)
meta = meta.iloc[:10000, :] # 메모리 문제로 10000 개만 실행

In [ ]:
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
def get_genres(x):
  result = []
  x = eval(x)
  for i in x:
    result.append(i['name'])
  return result

meta['clean_genres'] = meta['genres'].apply(get_genres)

In [ ]:
id2title = {}
title2id =  {}

for id, tilte in enumerate(meta['title']):
  id2title[id] = tilte
  title2id[tilte] = id

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(meta['clean_genres'].values).transform(meta['clean_genres'].values)

In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df.iloc[:, :] = df.values.astype(int)
df.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
def get_sim_tilte(sim_scores, idx, top = 10):
  sim_scores = [(i, c) for i, c in enumerate(sim_scores[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
  sim_scores = [(id2title[i], score) for i, score in sim_scores[0:top]]
  return sim_scores

In [ ]:
idx = title2id['Spider-Man 2']

In [ ]:
# 내적
dot_scores = np.dot(df.values, df.values.T)
get_sim_tilte(dot_scores, idx)

[('Mighty Morphin Power Rangers: The Movie', 3),
 ('Last Action Hero', 3),
 ('The Shadow', 3),
 ('Highlander', 3),
 ('Turbo: A Power Rangers Movie', 3),
 ('The Fifth Element', 3),
 ('Conan the Barbarian', 3),
 ('Spawn', 3),
 ('Kull the Conqueror', 3),
 ('The Borrowers', 3)]

In [ ]:
# 피어슨 유사도
pearson_scores = df.T.corr().values
get_sim_tilte(pearson_scores, idx)

[('Conan the Destroyer', 1.0),
 ('The Scorpion King', 1.0),
 ('Reign of Fire', 1.0),
 ('Sword of the Valiant: The Legend of Sir Gawain and the Green Knight', 1.0),
 ('Hawk the Slayer', 0.8401680504168053),
 ('Mothra vs. Godzilla', 0.8401680504168053),
 ('Pokémon Heroes: Latios and Latias', 0.8401680504168053),
 ('Mortal Kombat', 0.7934920476158719),
 ('Cutthroat Island', 0.7934920476158717),
 ('Shogun Assassin', 0.7934920476158717)]

In [ ]:
# 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity
cosine_scores = cosine_similarity(df, df)
get_sim_tilte(cosine_scores, idx)

[('Highlander', 1.0000000000000002),
 ('Conan the Barbarian', 1.0000000000000002),
 ('Red Sonja', 1.0000000000000002),
 ('The Mummy', 1.0000000000000002),
 ('The 13th Warrior', 1.0000000000000002),
 ('The Mummy Returns', 1.0000000000000002),
 ('Krull', 1.0000000000000002),
 ('The Lord of the Rings: The Fellowship of the Ring', 1.0000000000000002),
 ('Conan the Destroyer', 1.0000000000000002),
 ('The Scorpion King', 1.0000000000000002)]

In [ ]:
# 유클리드 거리
from sklearn.metrics.pairwise import euclidean_distances
euclidean_scores = euclidean_distances(df, df)
get_sim_tilte(euclidean_scores, idx)

[('The Advocate', 3.1622776601683795),
 ('Radioland Murders', 3.0),
 ('Deathtrap', 3.0),
 ('The Forgotten', 3.0),
 ('The Hound of the Baskervilles', 3.0),
 ('The Fourth Man', 3.0),
 ('Disclosure', 2.8284271247461903),
 ("Widows' Peak", 2.8284271247461903),
 ('To Catch a Thief', 2.8284271247461903),
 ('The Return of Martin Guerre', 2.8284271247461903)]

In [ ]:
# 자카르도 거리
from sklearn.metrics.pairwise import pairwise_distances
pairwise_scores = pairwise_distances(df.values, metric='jaccard')
get_sim_tilte(pairwise_scores, idx)

[('Toy Story', 1.0),
 ('Grumpier Old Men', 1.0),
 ('Waiting to Exhale', 1.0),
 ('Father of the Bride Part II', 1.0),
 ('Sabrina', 1.0),
 ('The American President', 1.0),
 ('Dracula: Dead and Loving It', 1.0),
 ('Nixon', 1.0),
 ('Casino', 1.0),
 ('Sense and Sensibility', 1.0)]

## TF-IDF

영화의 개요를 기반으로 유사한 영화를 추천

input = 행 - 영화, 열 - 토근화된 영화의 개요, 값 - 빈도 수

output = 개요가 비슷한 영화를 추천

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
movie_paths = '/content/drive/MyDrive/추천/data/movies/'

meta = pd.read_csv(movie_paths + 'movies_metadata.csv', low_memory=False)
meta = meta[meta['overview'].notnull()].reset_index(drop=True)
meta = meta.iloc[:10000, :] # 메모리 문제로 10000 개만 실행
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(meta['overview'])
tfidf_matrix.shape

(10000, 32382)

In [ ]:
id2title = {}
title2id =  {}

for id, tilte in enumerate(meta['title']):
  id2title[id] = tilte
  title2id[tilte] = id

In [ ]:
idx = title2id['Spider-Man 2']

In [ ]:
def get_sim_tilte(sim_scores, idx, top = 10):
  sim_scores = [(i, c) for i, c in enumerate(sim_scores[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
  sim_scores = [(id2title[i], score) for i, score in sim_scores[0:top]]
  return sim_scores

In [ ]:
# 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity
cosine_scores = cosine_similarity(tfidf_matrix, tfidf_matrix)
get_sim_tilte(cosine_scores, idx)

[('Spider-Man', 0.20007530060992013),
 ('The Toxic Avenger Part III: The Last Temptation of Toxie',
  0.16328816357026565),
 ('The Outlaw', 0.12839369519033741),
 ('Arachnophobia', 0.12836860018184662),
 ('The Hit', 0.12140730714188261),
 ('Superman', 0.1106938437487527),
 ('The Haunted Palace', 0.10989800587571551),
 ('Hero at Large', 0.1070940460223121),
 ('Permanent Vacation', 0.10259252529665207),
 ('A Small Circle of Friends', 0.10198030758595306)]

In [ ]:
# 유클리드 거리
from sklearn.metrics.pairwise import euclidean_distances
euclidean_scores = euclidean_distances(tfidf_matrix, tfidf_matrix)
get_sim_tilte(euclidean_scores, idx)

[('The Quarry', 1.4142135623730954),
 ("Recess: School's Out", 1.4142135623730954),
 ('Moby Dick', 1.4142135623730954),
 ('Mr. Hobbs Takes a Vacation', 1.4142135623730954),
 ('Mrs. Parkington', 1.4142135623730954),
 ('The Loved One', 1.4142135623730954),
 ("National Lampoon's Gold Diggers", 1.4142135623730954),
 ('Toy Story', 1.4142135623730951),
 ('Grumpier Old Men', 1.4142135623730951),
 ('Tom and Huck', 1.4142135623730951)]

## Word2Vec

사용자가 영화를 본 순서에 맞춰서 영화를 추천

input = 사용자가 영화를 본 순서 데이터

output = 현재 사용자가 보는 영화를 기준으로 비슷한 순서의 영화를 추천

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
movie_paths = '/content/drive/MyDrive/추천/data/movies/'

movie = pd.read_csv(movie_paths + "ratings.csv")
meta = pd.read_csv(movie_paths + 'movies_metadata.csv', low_memory=False)
meta = meta.rename(columns={'id':'movieId'})

In [ ]:
movie.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie, meta[['movieId', 'original_title']], on='movieId')
movie.head()

,userId,movieId,rating,timestamp,original_title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III


In [ ]:
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head()

,unique
userId,
1,"[Rocky III, Greed, American Pie, My Tutor, Jay..."
2,"[The Dark, 2001: A Space Odyssey, Trois couleu..."
3,"[Trois couleurs : Rouge, Terminator 3: Rise of..."
4,"[Rocky III, American Pie, My Tutor, Lost in Tr..."
5,"[Jay and Silent Bob Strike Back, 48 Hrs., Batm..."


In [ ]:
sentence = []
for user_sentence in agg['unique'].values:
    sentence.append(list(map(str, user_sentence)))

In [ ]:
 from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence, size=20, window = 5, min_count=1, workers=4, iter=200, sg=1) # skip-gram 방식

In [ ]:
embedding_model.wv.most_similar(positive=['Spider-Man 2'], topn=10)

[('Snow Cake', 0.9150896072387695),
 ("Mr. Magorium's Wonder Emporium", 0.8948473334312439),
 ('Jungle Fever', 0.862638533115387),
 ('Poodle Springs', 0.8605114817619324),
 ('Cruel Intentions 3', 0.8416417241096497),
 ('Death Machine', 0.8409174084663391),
 ('Hook', 0.8406563401222229),
 ('Prelude to a Kiss', 0.8400174975395203),
 ('Johnny Handsome', 0.8371172547340393),
 ('Star Trek VI: The Undiscovered Country', 0.8329635262489319)]

# 데이터 출처
- https://www.kaggle.com/chocozzz/t-academy-recommendation2


# 참고자료

- https://developers.google.com/machine-learning/recommendation?hl=ko

- https://tacademy.skplanet.com/live/player/onlineLectureDetail.action?seq=194

- https://www.youtube.com/watch?v=1fhNX_JFaow

- https://brunch.co.kr/@kakao-it/72#comment

- https://www.youtube.com/watch?v=h6vePourB7E&list=LL&index=1&t=15s

- https://ratsgo.github.io/generative%20model/2018/01/27/VAE/

- https://github.com/GunhoChoi/PyTorch-FastCampus/blob/master/08_Autoencoder/3_Variational_Autoencoder.ipynb

- https://ko-kr.facebook.com/groups/TensorFlowKR/permalink/524873987853664/

- https://doheelab.github.io/recommender-system/ncf_mlp/